# **Data Cleaning Notebook**

## Objectives

* Evaluate missing data
* Clean Data

## Inputs

* inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv

## Outputs

* Generate cleaned Test and Train sets 

## Conclusions

* Data cleaning pipeline
* Drop variable ['EnclosedPorch']

---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load collected data

In [ ]:
import pandas as pd
df = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv")
df.head()

# Data Exploration

First we need to find the values with missing data, store in an array and create a Pandas profiling report.
This shows 9 variables with missing data, the total of each can be seen in the report.

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

In [ ]:
from ydata_profiling import ProfileReport
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

for col in ['2ndFlrSF', 'BedroomAbvGr', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea', 'WoodDeckSF']:
  sns.histplot(data=df, x=col, kde=True)
  plt.show()
  print('\n')

# Correlation and PPS Analysis

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps


def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r', annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):

    print("\n")
    print("* Analyse how the target variable for your ML models are correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

Correlations and Powerscore

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

Display Heatmaps

In [ ]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman, 
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.4, PPS_Threshold =0.2,
                  figsize=(12,10), font_annot=10)

---

# Data Cleaning

# Assessing Missing Data Levels

* Custom function to display missing data levels in a DataFrame, it shows the absolute levels, relative levels and data type.

In [12]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                   "PercentageOfDataset": missing_data_percentage,
                                   "DataType": df.dtypes}
                                    )
                          .sort_values(by=['PercentageOfDataset'], ascending=False)
                          .query("PercentageOfDataset > 0")
                          )

    return df_missing_data

Check missing data levels for the collected dataset.

In [ ]:
EvaluateMissingData(df)

# Dealing With Missing Data

Analyses and visualizes the effects of data cleaning methods on specified variables.

In [14]:
def DataCleaningEffect(df_original, df_cleaned, variables_applied_with_method):
      
    # Initialize plot counter
    flag_count = 1  

    # Separate categorical and numerical variables for ease of plotting
    categorical_vars = [var for var in variables_applied_with_method if df_original[var].dtype == 'object']
    numerical_vars = [var for var in variables_applied_with_method if df_original[var].dtype != 'object']

    print("\n=====================================================================================")
    print("* Distribution Effect Analysis After Data Cleaning Method in the following variables:\n")
    print(f"{variables_applied_with_method}\n")

    # Plot for categorical variables (bar plots)
    for var in categorical_vars:
        df1 = pd.DataFrame({"Type": "Original", "Value": df_original[var]})
        df2 = pd.DataFrame({"Type": "Cleaned", "Value": df_cleaned[var]})
        df_combined = pd.concat([df1, df2], axis=0)

        plt.figure(figsize=(15, 5))
        sns.countplot(data=df_combined, x="Value", hue="Type", palette=['#432371', "#FAAE7B"])
        plt.title(f"Distribution Plot {flag_count}: {var} (Categorical Variable)")
        plt.xticks(rotation=90)
        plt.legend(title="Dataset")
        plt.xlabel(var)
        plt.ylabel("Count")
        plt.show()

        flag_count += 1

    # Plot for numerical variables (histograms)
    for var in numerical_vars:
        plt.figure(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True, element="step")
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True, element="step")
        plt.title(f"Distribution Plot {flag_count}: {var} (Numerical Variable)")
        plt.legend(title="Dataset")
        plt.xlabel(var)
        plt.ylabel("Frequency")
        plt.show()

        flag_count += 1

Numerical Values

In [ ]:
variables_method = ['2ndFlrSF', 'BedroomAbvGr', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea', 'WoodDeckSF', 'EnclosedPorch']
variables_method

Using the Median imputation method on all numerical variables with missing data with the Data Cleaning Effect function

In [16]:
from feature_engine.imputation import MeanMedianImputer

imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)
df_method = imputer.fit_transform(df)

In [ ]:
DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

Categorical Values

In [ ]:
variables_method = ['GarageFinish', 'BsmtFinType1']
variables_method

In [19]:
from feature_engine.imputation import CategoricalImputer

imputer = CategoricalImputer(imputation_method='missing',fill_value='Missing',
                             variables=variables_method)

df_method = imputer.fit_transform(df)

In [ ]:
DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

## Data Cleaning Summary

- Drop - EnclosedPorch and WoodDeckSF as these have over 88% missing values
- Median Inputing - LotFrontage, GarageYrBlt, MasVnrArea, BedroomAbvGr, 2ndFlrSF. 
- Categorical Imputing to 'missing' - GarageFinish and BsmtFinType1

# Split Train and Test Models

In [ ]:
from sklearn.model_selection import train_test_split

TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

### Drop Variables

We are dropping EnclosedPorch and WoodDeckSF as these have over 88% missing values.

In [ ]:
variables_method = ['EnclosedPorch', 'WoodDeckSF' ]

print(f"* {len(variables_method)} variables to drop \n\n"
    f"{variables_method}")

In [25]:
from feature_engine.selection import DropFeatures

imputer = DropFeatures(features_to_drop=variables_method)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

In [ ]:
EvaluateMissingData(TrainSet)

### Median Imputing

In [ ]:
variables_method = ['LotFrontage', 'GarageYrBlt', 'MasVnrArea', 'BedroomAbvGr', '2ndFlrSF' ]
print(f"* {len(variables_method)}  median\n\n"
    f"{variables_method}")

In [30]:
imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)
imputer.fit(TrainSet)
df_method = imputer.transform(TrainSet)

In [ ]:
df_method.head(5)

In [37]:
imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

In [ ]:
EvaluateMissingData(TrainSet)

### Categorical Imputing

In [ ]:
variables_method = [ 'GarageFinish', 'BsmtFinType1' ]
print(f"* {len(variables_method)}  categorical\n\n"
    f"{variables_method}")

In [40]:
from feature_engine.imputation import CategoricalImputer

imputer = CategoricalImputer(imputation_method='missing',fill_value='Missing', variables=variables_method)
imputer.fit(TrainSet)
df_method = imputer.transform(TrainSet)

In [ ]:
df_method.head(5)

In [42]:
imputer = CategoricalImputer(imputation_method='missing',fill_value='Missing', variables=variables_method)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

In [ ]:
EvaluateMissingData(TrainSet)

---

# Push files to Repo

In [44]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
  print(e)


## Train Set

In [45]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

## Test set

In [46]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)